ALMECIJA César

FONTAINE Florestan

# Projet d'optimisation - Au fil de l'eau

## Préambule : imports et définitions des constantes

In [1]:
# Imports
from casadi import *
import numpy as np

In [2]:
# Constantes
l = 7
m = 5
mr = 2
md = 3

# Données du problème
A = np.array([[-1,-1,0,0,0,0,0],
              [0,0,-1,-1,0,0,0],
              [1,0,0,1,-1,0,-1],
              [0,1,0,0,1,-1,0],
              [0,0,1,0,0,1,1]])
r = np.array([100,10,1000,100,100,10,1000])

# Conditions initiales
pr = np.array([105,104])
fd = np.array([0.08,1.3,0.13])

## Partie I

## Question 1 : justification du coût

L'objectif de cette étude est de connaître des informations physiques sur le réseau : les pressions, les flux et les débits. Ils correspondent donc à la réalité physique sous-jacente de notre réseau. Celle-ci se traduit par le respect des équations $(2)$ et $(4)$, qui peuvent s'écrire :

$$
\left\{
\begin{array}{rcl}
\|Aq-f\|^2&=&0\\
\|r\bullet q \bullet | q | - A^Tp\|^2 & = & 0
\end{array}
\right.
$$

Ainsi, pour s'approcher de la réalité physique du réseau, il faut que ces deux équations soient respectées. On cherche donc à minimiser les valeurs $\|Aq-f\|^2$ et $\|r\bullet q \bullet | q | - A^Tp\|^2$, puisque leur minimum vaut justement 0.

Comme ces deux valeurs sont positives, en minimisant leur somme on s'assure de minimmiser $\|Aq-f\|^2$ et $\|r\bullet q \bullet | q | - A^Tp\|^2$ individuellement (si la somme est petite, alors chaque terme est au moins aussi petit).

**La valeur $\|Aq-f\|^2 + \|r\bullet q \bullet | q | - A^Tp\|^2$ est donc bien un coût, puisque c'est une fonction que l'on cherche à minimiser pour s'approcher de la réalité physique du réseau que l'on modélise.**

## Question 2 : formulation du premier problème d'optimisation

### $\bullet$ Choix de $z$

On choisit $z$ comme un vecteur de taille $n=2m+l$, qui s'écrit :

$$
z =
\left(
\begin{array}{c}
p\\
f\\
q
\end{array}
\right)
$$

C'est-à-dire que :
* ses $m$ **premières coordonnées correspondent aux pressions** (plus précisemment, les $m_r$ premières correspondent aux *pressions aux réservoirs* et les $m_d$ suivantes aux *pressions aux demandes*).
* ses $m$ **coordonnées suivantes correspondent aux flux** (même remarque)
* enfin, ses $l$ **dernières coordonnées correspondent aux débits**.

### $\bullet$ Choix des contraintes égalités $c_{eq}$

La contrainte $c_{eq}$ est une contrainte qui impose les "conditions initiales" : dans notre cas, il s'agit des informations que l'on connaît. Plus précisemment, il s'agit de $p_r$ et de $f_d$. Ainsi, cela justifie la forme de $c_{eq}$ suivante :

$$
c_{eq}:
\left\{
\begin{array}{rcl}
\mathbb{R}^n & \longrightarrow & \mathbb{R}^m\\
z =
\left(
\begin{array}{c}
p_r\\
p_d\\
f_r\\
f_d\\
q
\end{array}
\right) & \longmapsto &
\left(
\begin{array}{c}
p_r - p_{r, connu}\\
f_d - f_{d, connu}
\end{array}
\right)
\end{array}
\right.
$$

### $\bullet$ Choix du coût $f(z)$

On reprend la fonction coût de la question précédente. $r$ et $A$ sont considérées comme des données du problème :

$$
f:
\left\{
\begin{array}{rcl}
\mathbb{R}^n & \longrightarrow & \mathbb{R}\\
z =
\left(
\begin{array}{c}
p\\
f\\
q
\end{array}
\right)& \longmapsto & \|Aq-f\|^2 + \|r\bullet q \bullet | q | - A^Tp\|^2
\end{array}
\right.
$$

## Question 3 : justification du deuxième problème d'optimisation

Dans cette démarche, l'inconnue est constituée des débits $q$ et de $f_r$. Elles ne correspondent pas à des données du problème (qui sont $p_r$ et $f_d$), donc il est normal que les données n'interviennent pas dans les contraintes.

Concernant le respect des équations physiques :
* l'une d'entre elles est directement spécifiée dans les contraintes (l'équation $(2)$), ce qui explique qu'elle soit une condition de stationnarité.
* l'autre (l'équation $(4)$) est dans les conditions de stationnarité admises. Atteindre, ou s'approcher du minimum, implique donc de respecter l'équation $(4)$.

Ainsi, résoudre ce problème fournit une **solution physiquement réaliste**. Sachant qu'elle prend en compte les données du problème ($f_d$ est contenu dans la contrainte, et $p_r$ est dans le coût), c'est une **bonne démarche pour résoudre ce problème.**

## Question 4 : comment trouver $p$ à l'aide du deuxième problème ?

Résoudre ce problème d'optimisation nous fournit $q$. Or, l'équation $(4)$ s'écrit :
$$
A^Tp=r\bullet q \bullet |q|
$$
Pour trouver $p$, on aimerait utiliser $A^{T^{-1}}$. Cependant, $A$ n'est pas carrée, donc *a fortiori* non-inversible. On opte donc pour une résolution numérique, en résolvant le problème d'optimisation suivant, où $r$, $q$ et $A$ sont des données :

$$
\min_{p \in \mathbb{R}^{m}, p_r=p_{r, donne}}\|A^Tp-r\bullet q \bullet |q|\|
$$

Il s'agit de la méthode des moindres carrées :
* lorsque $A$ est de rang plein (ce qui n'est pas le cas par la suite), on dispose d'une solution exacte vue en TD
* autrement, on passe par une résolution numérique (ce que l'on fera par la suite), en imposant la contrainte correspondant aux conditions initiales $p_r$.

## Question 5 : comparaison des deux problèmes d'optimisation

Voici une comparaison sur divers aspects. Ceux-ci seront rediscutés à la question 9.

### $\bullet$ Respect de la réalité physique

On rappelle que **l'objectif de la résolution de ces deux problèmes d'optimisation est de trouver la réalité physique sous-jacente du réseau**. Il est donc indispensable que la méthode de résolution assure de trouver une solution physiquement réaliste. Cela se traduit par le respect des équations $(2)$ et $(4)$.

Nous avons vu que la première méthode avait justement pour objectif de minimiser le "non-respect" de ces équations. La solution trouvée sera donc proche de la vérité physique.

Cependant, la deuxième méthode contient l'équation $(2)$ dans ses contraintes. On est donc assurés que **celle-ci sera respectée, et pas seulement approximée**. L'équation $(4)$ est une condition de stationnarité : en nous approchant du minimum, on trouvera une solution proche de cette condition.

Pour résumer :
* la deuxième méthode nous assure qu'une des deux conditions sera respectée. L'autre est approximée.
* dans la première méthode, les deux conditions sont approximées.

### $\bullet$ Conditions initiales

La première méthode contient les conditions initiales dans ses contraintes : on est donc assurés qu'elles seront respectées.

Dans la deuxième méthode :
* $f_d$ n'est pas dans les contraintes : c'est une donnée et sa valeur est simplement utilisée.
* $p_r$ est dans la contrainte permettant de retrouver $p$.

Dans tous les cas, on est donc assurés que les conditions initiales seront respectées.

### $\bullet$ A propos des pressions

Dans la deuxième méthode, pour trouver $p$, on doit résoudre deux problèmes d'optimisation : le problème $(7)$ et la méthode des moindres carrés (où l'on ne sait même pas s'il y a existence ou unicité des solutions). Il se peut donc que la deuxième méthode ne nous donne pas de solution en pression satisfaisante.

Ce phénomène n'intervient pas dans le premier problème.

## Partie II

## Question 6 : méthodes de résolution pour les deux problèmes d'optimisation

On peut dans les deux cas envisager d'utiliser une version modifiée de l'algorithme d'Arrow-Hurwicz où les conditions sont doublées et de signe différents pour avoir l'égalité, et où gradients et jacobiens sont approchés.
Une autre solution consisterait à implémenter l'algorithme d’élimination des variables pour contraintes linéaires.

On peut aussi utiliser une méthode primal-dual de point intérieur utilisant des recherches basées sur des méthode de filtre comme ceux de Fletcher et Leyffer. 
Une telle méthode fait intervenir les multiplicateurs de Lagrange une fonction barrière :
$$
B(x,\mu) = f(x) + \mu\sum_{i=1}^mlog(c_i(x))
$$
Où $\mu$ est un scaliare positif et $c_i$ les contraintes. En annulant le gradient de la fonction barrière, on s'assure que $f(x)$ réside dans l'espace de l'opposé des gradients des contraintes, ce qui se rapproche des conditions de KKT.

Autrement dit, on peut utiliser la librairie IPOPT (Interior Point OPTimizer).


Dans la pratique, on préférera uriliser la librairie IPOPT déjà implémentée par Casadi pour simplifier les calculs

## Question 7 : algorithme de résolution du premier problème d'optimisation

### $\bullet$ Implémentation de l'algorithme

On choisit d'utiliser le module `casadi` avec le logiciel d'optimisation `ipopt`.

In [3]:
opti1 = casadi.Opti()

# Récupération de la variable z
n = 2*m + l
z = opti1.variable(n)

# Définition de la fonction coût
f = casadi.norm_2(casadi.mtimes(A,z[2*m:])-z[m:2*m])**2 + casadi.norm_2(r*z[2*m:]*casadi.fabs(z[2*m:]) - casadi.mtimes(A.T,z[0:m]))**2
opti1.minimize(f)

# Définition des contraintes égalités
opti1.subject_to(z[0:mr] == pr)
opti1.subject_to(z[m+mr:2*m] == fd)

# Définition de la valeur initiale
z0 = np.ones(n)
opti1.set_initial(z,z0)

# Définition du logiciel de résolution
opti1.solver('ipopt')

In [4]:
# Résolution
sol1 = opti1.solve()


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        5
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      128

Total number of variables............................:       17
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equal

In [5]:
# Affichage des résultats

zsol = sol1.value(z)

p1 = zsol[:m]
f1 = zsol[m:2*m]
q1 = zsol[2*m:]

print("\n==Affichage des résultats (problème 1)==\n")
print("-->Pressions :")
print("    - pressions aux réservoirs :", p1[0:mr])
print("    - pressions aux demandes   :", p1[mr:])
print("-->Flux :")
print("    - flux aux réservoirs      :", f1[:mr])
print("    - flux aux demandes        :", f1[mr:])
print("-->Débits :")
print(q1)
print("\n==Fin de l'affichage des résultats==\n")


==Affichage des résultats (problème 1)==

-->Pressions :
    - pressions aux réservoirs : [105. 104.]
    - pressions aux demandes   : [108.69167351 114.93646695 114.90807311]
-->Flux :
    - flux aux réservoirs      : [-1.18895557 -0.32104443]
    - flux aux demandes        : [0.08 1.3  0.13]
-->Débits :
[ 0.19213728  0.99681829  0.10444172  0.21660271  0.24989585 -0.05328587
  0.07884415]

==Fin de l'affichage des résultats==



### $\bullet$ Quelques commentaires sur les résultats :

Remarquons que :
* les conditions initiales sont bien respectées (pressions aux réservoirs et flux aux demandes) (conformément à ce que l'on avait commenté à la question 5)
* les flux calculés aux réservoirs sont négatifs, ce qui correspond à la convention choisie.

Vérifions que les conditions physiques (*ie* les équations $(2)$ et $(4)$) sont respectées :

In [6]:
print("-->Valeur de ||Aq-f||        :", np.linalg.norm(np.dot(A, q1) - f1))
print("-->Valeur de ||A^T - r*q*q|| :", np.linalg.norm(np.dot(A.T, p1) - r*q1*np.abs(q1)))

-->Valeur de ||Aq-f||        : 1.156783582076316e-09
-->Valeur de ||A^T - r*q*q|| : 3.726666237877904e-09


On peut considérer qu'elles sont respectées (les valeurs sont petites devant les ordres de grandeur en jeu). La solution est donc physiquement admissible.

Ces résultats seront davantage discutés, et mis en regard avec la méthode de résolution suivante, à la question 9.

## Question 8 : algorithme de résolution du deuxième problème d'optimisation

### $\bullet$ Implémentation de l'algorithme

On choisit également d'utiliser le module `casadi` avec le logiciel d'optimisation `ipopt`.

In [7]:
opti2 = casadi.Opti()

# Récupération de la variable y=(q, f_r)
n = l + mr
y = opti2.variable(n)
    
# Définition de la fonction coût
f = (1/3)*casadi.dot(y[0:l],r*y[0:l]*casadi.fabs(y[0:l])) + casadi.dot(pr,y[l::])
opti2.minimize(f)

# Définition de la contrainte égalité
opti2.subject_to(casadi.mtimes(A,y[0:l]) - (casadi.vertcat(y[l::],fd))== 0)

# Définition de la valeur initiale
y0 = np.ones(n)
opti2.set_initial(y,y0)

# Définition du logiciel de résolution
opti2.solver('ipopt')

In [8]:
# Résolution du problème 2 (il reste à trouver p)
sol2 = opti2.solve()
ysol = sol2.value(y)

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       37
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        7

Total number of variables............................:        9
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        5
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

In [9]:
# Problème suivant : trouver p (moindres carrés)
opti3 = casadi.Opti()

# Récupération de la variable p
n = m
p = opti3.variable(n)
    
# Définition de la fonction coût
q = ysol[0:l]
b = r*q*casadi.fabs(q)
f = casadi.norm_2(casadi.mtimes(A.T,p)-b)
opti3.minimize(f)

# Définition de la contrainte égalité
opti3.subject_to(p[0:mr] == pr)

# Définition de la valeur initiale
p0 = np.ones(n)
opti3.set_initial(p,p0)

# Définition du logiciel de résolution
opti3.solver('ipopt')

In [10]:
# Résolution du problème permettant de trouver p
sol3 = opti3.solve()
psol = sol3.value(p)

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        2
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       15

Total number of variables............................:        5
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        2
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

In [11]:
# Affichage des résultats

p2 = psol
f2 = np.concatenate( (ysol[l:], fd) )
q2 = ysol[:l]

print("\n==Affichage des résultats (problème 2)==\n")
print("(les éléments avec * indiquent qu'ils n'ont pas été calculés et qu'il s'agit de données)")
print("-->Pressions :")
print("    - pressions aux réservoirs :", p2[:mr])
print("    - pressions aux demandes   :", p2[mr:])
print("-->Flux :")
print("    - flux aux réservoirs      :", f2[:mr])
print("    - flux aux demandes*       :", f2[mr:])
print("-->Débits :")
print(q2)
print("\n==Fin de l'affichage des résultats==\n")


==Affichage des résultats (problème 2)==

(les éléments avec * indiquent qu'ils n'ont pas été calculés et qu'il s'agit de données)
-->Pressions :
    - pressions aux réservoirs : [105. 104.]
    - pressions aux demandes   : [108.47504205 114.56490545 114.0466566 ]
-->Flux :
    - flux aux réservoirs      : [-1.22716618 -0.28283382]
    - flux aux demandes*       : [0.08 1.3  0.13]
-->Débits :
[ 0.21154295  1.01562323  0.09641917  0.18641465  0.24165809 -0.04271868
  0.07629951]

==Fin de l'affichage des résultats==



### $\bullet$ Quelques commentaires concernant les résultats :

Remarquons à nouveau que :
* les conditions initiales sont bien respectées (pressions aux réservoirs et flux aux demandes) (conformément à ce que l'on avait commenté à la question 5)
* les flux calculés aux réservoirs sont négatifs, ce qui correspond à la convention choisie.

Vérifions à nouveau que les conditions physiques (*ie* les équations $(2)$ et $(4)$) sont respectées :

In [12]:
print("-->Valeur de ||Aq-f||          :", np.linalg.norm(np.dot(A, q2) - f2))
print("-->Valeur de ||A^T*p - r*q*q|| :", np.linalg.norm(np.dot(A.T, p2) - r*q2*np.abs(q2)))

-->Valeur de ||Aq-f||          : 2.237726045655905e-16
-->Valeur de ||A^T*p - r*q*q|| : 1.8708286936372296


On peut à nouveau considérer que la solution est physiquement réalisable (l'erreur est faible devant les valeurs de pression).

Ces résultats seront davantage discutés, et mis en regard avec la méthode de résolution précédente, à la question 9.

## Question 9 : comparaison des résultats, étude et commentaires

### $\bullet$ Comparaison des résultats

Calculons la différence relative de pressions, de flux et de débits obtenus (l'indice 1 correspond à la première méthode ; l'indice 2 à la deuxième) :

In [13]:
# On calcule les normes des vecteurs différence

p_diff = np.linalg.norm(p1-p2)/np.linalg.norm(p1)
f_diff = np.linalg.norm(f1-f2)/np.linalg.norm(f1)
q_diff = np.linalg.norm(q1-q2)/np.linalg.norm(q1)

print("||p1-p2||/||p1||=", p_diff)
print("||f1-f2||/||f1||=", f_diff)
print("||q1-q2||/||q1||=", q_diff)

||p1-p2||/||p1||= 0.003928440267500656
||f1-f2||/||f1||= 0.030067601165880777
||q1-q2||/||q1||= 0.0403867523366248


Nous remarquons que les différences sont de l'ordre du pourcent. Les négliger se discute, mais on pourra considérer qu'elles sont égales.

Nous n'avons, en théorie, prouvé aucune équivalence entre les deux méthodes ; on pouvait donc s'attendre à de telles différences.

Enfin, on remarque que les **signes restent identiques** (pour les valeurs algébriques de débits et de flux), ce qui montre que les deux méthodes fournissent un réseau qui fonctionne de la même manière.

### $\bullet$ Respect de la réalité physique

Ceci se mesure avec le respect des équations $(2)$ et $(4)$. Nous rappelons les ordres de grandeur obtenus précedemment sur $\|Aq-f\|$ et $\|A^Tp - r\bullet q \bullet |q|\|$ respectivement :

| Equation | Méthode 1 | Méthode 2|
|----------|-----------|----------|
| 2        |$10^{-9}$  |$10^{-16}$|
| 4        |$10^{-9}$  |$1$       |

Ces résultats correspondent à ce que l'on avait établi à la question 5 :
* la première méthode donne des résultats approchés
* la deuxième méthode donne une solution exacte pour $(2)$ ($10^{-16}$ peut être considéré comme une erreur de calcul sur des doubles) et approchée pour $(4)$.

Cependant, l'erreur commises sur $(4)$ par la deuxième méthode est très grande devant celle de la méthode 1, dû au calcul fait par un second algorithme. Et, de plus, la solution exacte de la méthode 2 n'apporte qu'une correction minime (une erreur de $10^{-9}$ est déjà largement acceptable, étant donnés les ordres de grandeurs en jeu).

Ainsi, **en ce qui concerne le respect de la réalité physique, on préfèrera utiliser la première méthode.**

### $\bullet$ Respect des conditions initiales

Comme on s'y attendait (cf. question 5), les pressions $p_r$ et les flux $f_d$ rentrés dans les problèmes d'optimisation permettent dans les deux cas d'avoir des pressions et des flux qui respectent ces données.

### $\bullet$ Nombre d'itérations

Voici un tableau récapitulant le nombre d'exécution pour chaque méthode :

|Méthode 1|Méthode 2|
|---------|---------|
|106      |18       |

(Pour la méthode 2, on pris en compte les deux résolutions de 7 et 11 itérations respectivement)

Il est donc très avantageux de passer à la deuxième méthode, pour gagner un facteur 10 en temps de calcul. **En ce qui concerne le temps de calcul, on préfèrera utiliser la deuxième méthode.**

## Partie III

## Question 10 : étude du lagrangien du problème

Soit $\lambda$ fixé, et considérons :

$$
\Lambda :
\left\{
\begin{array}{rcl}
\mathbb{R}^{l+m_r} & \longrightarrow & \mathbb{R}\\
(q, f_r) & \longmapsto & \mathcal{L}(q, f_r, \lambda)
\end{array}
\right.
$$

Pour minimiser cette fonction, on calcule son gradient, son hessien en montrant qu'elle est convexe.

### $\bullet$ Calcul du gradient et du hessien

Le premier terme peut s'écrire :
$$
\frac{1}{3}\sum_{i=1}^lr_i|q_i|^3
$$

Sa dérivée partielle par rapport à $q_i$ s'écrit donc :
$$
\dfrac{3}{3}r_iq_i|q_i| = r_iq_i|q_i|
$$

Ceci fournit donc le premier terme du gradient :
$$
\left(
\begin{array}{c}
r \bullet q \bullet |q|\\
0
\end{array}
\right)
$$

Le deuxième terme du gradient est :
$$
\left(
\begin{array}{c}
0\\
p_r
\end{array}
\right)
$$

et le troisième terme :

$$
\left(
\begin{array}{c}
A^T\lambda\\
-\lambda_r
\end{array}
\right)
$$

Le gradient s'écrit donc :

$$
\left(
\begin{array}{c}
r \bullet q \bullet |q| + A^T\lambda\\
p_r -\lambda_r
\end{array}
\right)
$$

Puis, en calculant le hessien, on remarque que c'est une matrice diagonale qui s'écrit :
$$
\text{Diag}(2r_1|q_1|, 2r_2|q_2|,..., 2r_l|q_l|,0,...,0)
$$

Les termes diagonaux (*ie* les valeurs propres de la matrice) sont tous positifs, donc le hessien est positif : la fonction $\Lambda$ est donc **convexe**. Chercher ses minima est équivalent à chercher ses points stationnaires.

### $\bullet$ Recherche des points stationnaires

On résout $\nabla \Lambda = 0$. On trouve :
$$
\left\{
\begin{array}{rcl}
r \bullet q \bullet |q| + A^T\lambda&=&0\\
p_r&=&\lambda_r\\
\end{array}
\right.
$$

### $\bullet$ Expression de l'`argmin`

On peut donc en déduire une expression de l'`argmin` en $q$ :

$$
\forall 1\leq i\leq l, r_iq_i|q_i|=-(A^T\lambda)_i
$$
d'où :
$$
\forall 1\leq i\leq l, q_i=-\text{sgn}\left(\dfrac{1}{r_i}\left(A^T\lambda\right)_i\right) \cdot \sqrt{\left|\dfrac{1}{r_i}(A^T\lambda)_i\right|}
$$

### $\bullet$ Expression du `min` et reformulation du problème dual

En reprenant la première condition de stationnarité calculée précédemment, la valeur du lagrangien au minimum $\mathcal{L}^*$ peut se réecrire :
$$
\mathcal{L}^*=\frac{1}{3}q^T\left(-A^T\lambda\right) + p_r^Tf_r + \lambda^TAq - \lambda_d^Tf_d - \lambda_r^Tf_r
$$

Qui se simplifie en (en passant à la transposée dans le premier terme) :

$$
\mathcal{L}^*= p_r^Tf_r + \frac{2}{3}\lambda^TAq - \lambda_d^Tf_d - \lambda_r^Tf_r
$$

Puis en reprenant la deuxième condition de stationnarité :

$$
\mathcal{L}^*= \frac{2}{3}\lambda^TAq - \lambda_d^Tf_d
$$

Ainsi, le problème dual qui s'écrit initialement :

$$
\max_{\lambda}\min_{q, f_r}\mathcal{L}
$$

devient :

$$
\max_{\lambda}\frac{2}{3}\lambda^TAq(\lambda) - \lambda_d^Tf_d
$$

sous réserve que $\lambda_r=p_r$. La variable de décision n'est donc plus vraiment $\lambda$, mais $\lambda_d$:
$$
\max_{\lambda_d}\frac{2}{3}\lambda^TAq(\lambda) - \lambda_d^Tf_d
$$

En le transformant en problème de minimisation, cela devient :
$$
-\min_{\lambda_d}\lambda_d^Tf_d-\frac{2}{3}\lambda^TAq(\lambda)
$$

## Question 11 : implémentation de la méthode du lagrangien ; comparaison des résultats

Les cellules suivantes font les opérations suivantes :
* elles trouvent $\lambda$ (en résolvant le problème de minimisation précédent sur $\lambda_d$) ;
* elles en déduisent $q$ (d'après la relation sur l'`argmin` précédente) ;
* elles en déduisent $f$ (qui avait disparu des équations) de l'équation (2) ;
* La valeur de $p$ est calculée à l'aide de la méthode des moindres carrés (comme précédemment).

In [14]:
opti4 = casadi.Opti()

# Récupération de la variable z
n = md
z = opti4.variable(n)

# Définition de la fonction coût
ca = casadi.mtimes(A.T,casadi.vertcat(pr,z))/r
q = -casadi.sign(ca)*(casadi.sqrt(casadi.fabs(ca)))
g = (2/3)*casadi.dot(casadi.vertcat(pr,z),casadi.mtimes(A,q))-casadi.dot(z,fd)
opti4.minimize(-g)

# Aucune contrainte

# Définition de la valeur initiale (on ne peut pas prendre de condition np.ones(n), car le gradient de la racine divergerait, dans le calcul de q)
z0 = np.array([1, 1.1, 1.2])
opti4.set_initial(z,z0)

# Définition du logiciel de résolution
opti4.solver('ipopt')

In [15]:
# Résolution
sol4 = opti4.solve()

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        6

Total number of variables............................:        3
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

In [16]:
# Calcul des résultats :
# La valeur de q est déduite de celle de l'argmin
# La valeur de f (qui avait disparu des équations) est déduite de l'équation (2)
# La valeur de p est calculée à l'aide de la méthode des moindres carrés (comme précédemment)
lambda_sol = sol4.value(z)

ca = np.matmul(A.T,np.concatenate((pr,lambda_sol)))/r
q3 = -np.sign(ca)*(np.sqrt(np.abs(ca)))
f3 = np.matmul(A, q3)

In [17]:
# Problème suivant : trouver p (moindres carrés)
opti5 = casadi.Opti()

# Récupération de la variable p
n = m
p_lag = opti5.variable(n)
    
# Définition de la fonction coût
b = r*q3*casadi.fabs(q3)
f = casadi.norm_2(casadi.mtimes(A.T,p_lag)-b)
opti5.minimize(f)

# Définition de la contrainte égalité
opti5.subject_to(p_lag[0:mr] == pr)

# Définition de la valeur initiale
p_lag0 = np.ones(n)
opti5.set_initial(p_lag,p_lag0)

# Définition du logiciel de résolution
opti5.solver('ipopt')

In [18]:
# Résolution du problème permettant de trouver p
sol5 = opti5.solve()
p3 = sol5.value(p_lag)

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        2
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       15

Total number of variables............................:        5
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        2
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

In [19]:
# Affichage des résultats

print("\n==Affichage des résultats (problème 3)==\n")
print("-->Pressions :")
print("    - pressions aux réservoirs :", p3[0:mr])
print("    - pressions aux demandes   :", p3[mr:])
print("-->Flux :")
print("    - flux aux réservoirs      :", f3[:mr])
print("    - flux aux demandes        :", f3[mr:])
print("-->Débits :")
print(q3)
print("\n==Fin de l'affichage des résultats==\n")


==Affichage des résultats (problème 3)==

-->Pressions :
    - pressions aux réservoirs : [105. 104.]
    - pressions aux demandes   : [108.47504205 114.56490545 114.0466566 ]
-->Flux :
    - flux aux réservoirs      : [-1.22716618 -0.28283382]
    - flux aux demandes        : [0.08 1.3  0.13]
-->Débits :
[ 0.21154295  1.01562323  0.09641917  0.18641465  0.24165809 -0.04271868
  0.07629951]

==Fin de l'affichage des résultats==



In [20]:
print("-->Valeur de ||Aq-f||          :", np.linalg.norm(np.dot(A, q3) - f3))
print("-->Valeur de ||A^T*p - r*q*q|| :", np.linalg.norm(np.dot(A.T, p3) - r*q3*np.abs(q3)))

-->Valeur de ||Aq-f||          : 0.0
-->Valeur de ||A^T*p - r*q*q|| : 1.8708286933869718


Puisque l'on trouve f grâce à (2), il est normal que cette équation soit absolument vérifiée. En revanche, l'erreur sur (4), bien que toujours du domaine de l'acceptable, reste relativement élévée.

### $\bullet$ Comparaison des résultats

Voici la différence relative de pressions, de flux et de débits obtenus entre la méthode 2 et cette méthode (l'indice 2 correspond à la première méthode ; l'indice 3 à celle du lagrangien) :

In [21]:
# On calcule les normes des vecteurs différence

p_diff = np.linalg.norm(p2-p3)/np.linalg.norm(p2)
f_diff = np.linalg.norm(f2-f3)/np.linalg.norm(f2)
q_diff = np.linalg.norm(q2-q3)/np.linalg.norm(q2)

print("||p2-p3||/||p2||=", p_diff)
print("||f2-f3||/||f2||=", f_diff)
print("||q2-q3||/||q2||=", q_diff)

||p2-p3||/||p2||= 3.6352112913110143e-12
||f2-f3||/||f2||= 1.740609543553249e-11
||q2-q3||/||q2||= 1.893330508225475e-11


Nous remarquons que les différences sont négligeables. **Cette méthode fournit les mêmes solutions que la méthode 2**. Résoudre le problème dual revient donc au même que résoudre le problème original, *ie* que résoudre le problème primal. Le saut de dualité est donc nul.

Ces solutions respectent donc la réalité physique et les conditions initiales de la même manière, au détail près que la relation (2) est absolument vérifiée avec la méthode 3.

### $\bullet$ Nombre d'itérations

Il est pertinent de comparer le temps d'exécution de la méthode 2 et de celle du lagrangien (elles donnent les mêmes résultats) :

| Méthode 2 | Méthode 3 |
|-----------|-----------|
|18         |24         |

Elles sont du même ordre de grandeur, avec un léger avantage pour la deuxième méthode. Passer par le problème dual est donc légèrement plus lent.